In [1]:
# Init
import os
import sys

module_path = os.path.abspath(os.path.join("../src/simulicronalpha/"))
if module_path not in sys.path:
    sys.path.append(module_path)

# Imports
import random
import numpy as np
import pandas as pd
import warnings
import multiprocessing

from numpy import concatenate as c

In [2]:
from popSim import (
    generateTransposon,
    generatePopulation,
    calculateFitness,
    generateGenome,
    transposition,
    runSim,
    runBatch,
    recombination,
)

In [3]:
gen = generateGenome(
    numberOfInsertionSites=1000,
    numberOfChromosomes=1,
    baseRecombinationRate=0.0,
    baseSelection=1,
)
tr, gen, TEset = generateTransposon(
    genomeArray=gen,
    NumberOfTransposonInsertions=1,
    baseExcision=1.0,
)
pop = generatePopulation(
    tr,
    InsertIntoAll=True,
    NumberOfIndividual=10,
    NumberOfTransposonInsertions=1,
)

In [4]:
transposition(transposonMatrix=tr, genomeMatrix=gen, NumberOfTransposonInsertions=1, TEset=TEset, v1=pop[1][0], v2=pop[1][1])

([2],
 [1],
 array([[0, 0, 0, 0, 0, 0],
        ['1596646b7a5829336a01ea2eae0283', 430, 1.0, 1.0, 1, 1],
        ['8e1eff844bd431951b0b59cd2d888c', 912, 1.0, 1.0, 1, 1]],
       dtype=object),
 {1: {1, 2}})

In [6]:
v1=pop[:,0]

In [5]:
tr

array([[0, 0, 0, 0, 0, 0],
       ['3e8b6b15a9703e0fc091f055356079', 880, 1.0, 1.0, 1, 1],
       ['42b3864d26259a7f655b100535f46e', 570, 1.0, 1.0, 1, 1]],
      dtype=object)

In [5]:
pop

array([[0, list([1]), 2.718281828459045],
       [0, list([1]), 2.718281828459045],
       [0, list([1]), 2.718281828459045],
       [list([1]), 0, 2.718281828459045],
       [list([1]), 0, 2.718281828459045],
       [list([1]), 0, 2.718281828459045],
       [list([2]), 0, 2.718281828459045],
       [list([2]), 0, 2.718281828459045],
       [list([1]), 0, 2.718281828459045],
       [list([2]), 0, 2.718281828459045]], dtype=object)

In [6]:
tr

array([[0, 0, 0, 0, 0, 0],
       ['307ec65e58e9b57d01e4e26e6e3e35', 80, 1.0, 1.0, 1, 1],
       ['0d5e9ccc45470b382ffe682228b209', 368, 1.0, 1.0, 1, 1]],
      dtype=object)

In [7]:
TEset

{1: {1, 3}, 2: {2}}

In [5]:
v1 = np.asarray([True, True, False])
v2 = np.asarray([False, True, True])
v3 = np.asarray([False, True, False])

In [6]:
v1 & v2 & v3

array([False,  True, False])

In [ ]:
|